# Chord and Scale Analysis
Using the Spotify API, I can pull the dominant notes for any moment in a song. Using those dominant notes, I should be able to determine the chord and possibly the scale(s) and key of the song.

For these experiments, I'll use Oscar Peterson's "Hymn to Freedom." This starts with a slow, solo piano with reasonably simple chord voicings. Since I have the sheet music, I can also compare the dominant notes from the Spotify API with the actual music.

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pandasql as ps
import numpy as np

# # Spotify Credentials
import spot_creds

import sp_functions as spf

clid = spot_creds.client_id
secret = spot_creds.secret

In [2]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=clid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
track_uri = '1kGQzSasZr4HY5CzjHqCPG'
track_details = spf.extract_track_feat(track_uri)
track_details

,track_uri,track_name,artist_uri,artist_name,artist_pop,artist_genres,album,track_pop,explicit,acousticness,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1kGQzSasZr4HY5CzjHqCPG,Hymn To Freedom,spotify:artist:0ldU0QJm31y0d6f57R1G2A,Oscar Peterson Trio,58,"[bebop, contemporary post-bop, jazz, jazz trio]",Night Train (Expanded Edition),42,False,0.969,...,0.118,0.835,10,0.0845,-20.609,1,0.0451,83.893,4,0.198


In [28]:
# bars
bars = pd.DataFrame(audio_anal['bars'])
bars['bar_start'] = bars['start']
bars['bar_end'] = bars.start + bars.duration
bars['bar_number'] = bars.index
bars['bar_duration'] = bars['duration']
bars = bars[['bar_number','bar_start','bar_end','bar_duration','confidence']]
bars.head()

,bar_number,bar_start,bar_end,bar_duration,confidence
0,0,2.62965,5.68891,3.05926,0.149
1,1,5.68891,8.68213,2.99322,0.587
2,2,8.68214,11.64258,2.96044,0.243
3,3,11.64258,14.58020,2.93762,0.493
4,4,14.58020,17.47442,2.89422,0.527


In [23]:
# beats
beats = pd.DataFrame(audio_anal['beats'])
beats['beat_start'] = beats['start']
beats['beat_end'] = beats.start + beats.duration
beats.head()

,start,duration,confidence,beat_start,beat_end
0,0.28588,0.79861,0.999,0.28588,1.08449
1,1.08449,0.77496,0.093,1.08449,1.85945
2,1.85944,0.77021,0.546,1.85944,2.62965
3,2.62965,0.77573,0.588,2.62965,3.40538
4,3.40538,0.76913,0.374,3.40538,4.17451


In [31]:
sqlcode = '''
select bars.bar_number,
    bars.bar_start,
    bars.bar_end, 
    bars.bar_duration,
    beats.beat_start, 
    beats.beat_end
from bars
inner join beats
    on beats.beat_start >= bars.bar_start
    and beats.beat_end < bars.bar_end'''

newdf = ps.sqldf(sqlcode,locals())
newdf.head(12)

,bar_number,bar_start,bar_end,bar_duration,beat_start,beat_end
0,0,2.62965,5.68891,3.05926,2.62965,3.40538
1,0,2.62965,5.68891,3.05926,3.40538,4.17451
2,0,2.62965,5.68891,3.05926,4.17451,4.93437
3,1,5.68891,8.68213,2.99322,5.68891,6.44532
4,1,5.68891,8.68213,2.99322,6.44532,7.19832
5,1,5.68891,8.68213,2.99322,7.19832,7.94065
6,2,8.68214,11.64258,2.96044,8.68214,9.42341
7,2,8.68214,11.64258,2.96044,9.42340,10.16446
8,2,8.68214,11.64258,2.96044,10.16446,10.90461
9,3,11.64258,14.58020,2.93762,11.64258,12.38029


In [10]:
# sections
sections = pd.DataFrame(audio_anal['sections'])
sections.head()

,start,duration,confidence,loudness,tempo,tempo_confidence,key,key_confidence,mode,mode_confidence,time_signature,time_signature_confidence
0,0.00000,13.11770,1.000,-31.458,80.707,0.496,3,0.000,1,0.000,4,1.0
1,13.11770,28.74422,0.568,-28.628,83.235,0.422,10,0.248,1,0.397,4,1.0
2,41.86192,11.24689,0.468,-23.667,85.343,0.561,10,0.138,1,0.606,4,1.0
3,53.10881,21.01405,0.559,-25.527,86.027,0.520,5,0.133,1,0.579,4,1.0
4,74.12286,11.11655,0.448,-31.912,86.556,0.474,3,0.000,1,0.000,4,1.0


In [11]:
# segments
segments = pd.DataFrame(audio_anal['segments'])
segments.head()

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,pitches,timbre
0,0.00000,0.24785,0.000,-60.000,0.00000,-60.000,0.0,"[1.0, 0.999, 0.345, 0.262, 0.204, 0.151, 0.096...","[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ..."
1,0.24785,0.97615,1.000,-60.000,0.06727,-35.092,0.0,"[0.011, 0.016, 0.323, 0.01, 0.012, 0.329, 0.00...","[18.278, -120.569, -37.154, -20.836, 87.604, 6..."
2,1.22399,0.45469,0.006,-45.980,0.04209,-44.416,0.0,"[0.077, 0.071, 0.629, 0.198, 0.143, 0.5, 0.102...","[14.069, -68.136, -38.355, -25.376, 69.069, -4..."
3,1.67868,0.12671,0.113,-48.825,0.08638,-46.870,0.0,"[0.172, 0.168, 1.0, 0.079, 0.065, 0.527, 0.053...","[12.471, -67.385, -23.501, -53.955, 120.514, -..."
4,1.80540,0.83515,0.875,-46.953,0.09411,-37.293,0.0,"[0.113, 0.163, 0.997, 0.26, 0.129, 0.125, 1.0,...","[15.569, -107.861, -17.536, -69.326, 108.505, ..."


In [36]:
segments.iloc[4]['pitches']

[0.113,
 0.163,
 0.997,
 0.26,
 0.129,
 0.125,
 1.0,
 0.12,
 0.094,
 0.657,
 0.119,
 0.066]

In [4]:
audio_anal = spf.get_audio_analysis(track_uri)
audio_anal

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1573041945,
  'analysis_time': 123.82769,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 7353822,
  'duration': 333.50668,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.24785,
  'start_of_fade_out': 325.567,
  'loudness': -20.609,
  'tempo': 83.893,
  'tempo_confidence': 0.668,
  'time_signature': 4,
  'time_signature_confidence': 0.993,
  'key': 10,
  'key_confidence': 0.484,
  'mode': 1,
  'mode_confidence': 0.531,
  'code_version': 3.15,
  'echoprint_version': 4.12,
  'synch_version': 1.0,
  'rhythm_version': 1.0},
 'bars': [{'start': 2.62965, 'duration': 3.05926, 'confidence': 0.149},
  {'start': 5.68891, 'duration': 2.99322, 'confidence': 0.587},
  {'start': 8.68214, 'duration': 2.96044, 'confidence': 0.243},
  {'start': 11.642